In [3]:
# 查看当前挂载的数据集目录
!ls /home/aistudio/data/

data7983


In [4]:
# 查看个人持久化工作区文件
!ls /home/aistudio/work/

pretrained


!unzip data/data7983/cat_12.zip

!wget https://paddle-imagenet-models-name.bj.bcebos.com/ResNet101_vd_pretrained.tar

!tar -xvf ResNet101_vd_pretrained.tar -C work/pretrained

# Step-1

In [1]:
import os
import shutil
import paddle as paddle
import paddle.fluid as fluid
from paddle.fluid.param_attr import ParamAttr
import reader
import numpy as np
import pandas as pd

In [6]:
##生成test.txt
#filepath='cat_12/cat_12_test'
#pathlist=os.listdir(filepath)
#print(pathlist)
#with open('cat_12/test_list.txt','w+') as f:
    #for file_name in pathlist:
        #f.write(file_name+'\n')

In [2]:
a=pd.read_table('cat_12/train_list.txt',header=None)
print(a[1].value_counts())

11    180
9     180
7     180
5     180
3     180
1     180
10    180
8     180
6     180
4     180
2     180
0     180
Name: 1, dtype: int64


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [2]:
##处理数据集，将原有训练集按二八比例分割
with open('cat_12/train_list.txt','r') as f:
    with open('cat_12/train_split_list.txt','w+') as train:
        lines=f.readlines()
    with open('cat_12/train_split_list.txt','w+') as train:
        for i in range(0,2160,180):
            for line in lines[i:i+135]:
                train.write(line)
    with open('cat_12/val_split_list.txt','w+') as val:
        for i in range(0,2160,180):
            for line in lines[i+135:i+180]:
                val.write(line)

In [3]:
train=pd.read_table('cat_12/train_split_list.txt',header=None)
print(train[1].value_counts())
val=pd.read_table('cat_12/val_split_list.txt',header=None)
print(val[1].value_counts())

11    135
10    135
9     135
8     135
7     135
6     135
5     135
4     135
3     135
2     135
1     135
0     135
Name: 1, dtype: int64
11    45
10    45
9     45
8     45
7     45
6     45
5     45
4     45
3     45
2     45
1     45
0     45
Name: 1, dtype: int64


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
##获取猫数据
train_reader = paddle.batch(reader.train(), batch_size=16)
test_reader = paddle.batch(reader.val(), batch_size=16)

In [5]:
##定义残差网络
def resnet(input):
    def conv_bn_layer(input, num_filters, filter_size, stride=1, groups=1, act=None, name=None):
        conv = fluid.layers.conv2d(input=input,
                                   num_filters=num_filters,
                                   filter_size=filter_size,
                                   stride=stride,
                                   padding=(filter_size - 1) // 2,
                                   groups=groups,
                                   act=None,
                                   param_attr=ParamAttr(name=name + "_weights"),
                                   bias_attr=False,
                                   name=name + '.conv2d.output.1')
        if name == "conv1":
            bn_name = "bn_" + name
        else:
            bn_name = "bn" + name[3:]
        return fluid.layers.batch_norm(input=conv,
                                       act=act,
                                       name=bn_name + '.output.1',
                                       param_attr=ParamAttr(name=bn_name + '_scale'),
                                       bias_attr=ParamAttr(bn_name + '_offset'),
                                       moving_mean_name=bn_name + '_mean',
                                       moving_variance_name=bn_name + '_variance', )

    def shortcut(input, ch_out, stride, name):
        ch_in = input.shape[1]
        if ch_in != ch_out or stride != 1:
            return conv_bn_layer(input, ch_out, 1, stride, name=name)
        else:
            return input
            
    def bottleneck_block(input, num_filters, stride, name):
        conv0 = conv_bn_layer(input=input,
                              num_filters=num_filters,
                              filter_size=1,
                              act='relu',
                              name=name + "_branch2a")
        conv1 = conv_bn_layer(input=conv0,
                              num_filters=num_filters,
                              filter_size=3,
                              stride=stride,
                              act='relu',
                              name=name + "_branch2b")
        conv2 = conv_bn_layer(input=conv1,
                              num_filters=num_filters * 4,
                              filter_size=1,
                              act=None,
                              name=name + "_branch2c")

        short = shortcut(input, num_filters * 4, stride, name=name + "_branch1")

        return fluid.layers.elementwise_add(x=short, y=conv2, act='relu', name=name + ".add.output.5")

    depth = [3, 4, 23, 3]
    num_filters = [64, 128, 256, 512]

    conv = conv_bn_layer(input=input, num_filters=64, filter_size=7, stride=2, act='relu', name="conv1")
    conv = conv_bn_layer(input=input, num_filters=32, filter_size=3, stride=2, act='relu', name='conv1_1')
    conv = conv_bn_layer(input=conv, num_filters=32, filter_size=3, stride=1, act='relu', name='conv1_2')
    conv = conv_bn_layer(input=conv, num_filters=64, filter_size=3, stride=1, act='relu', name='conv1_3')
    conv = fluid.layers.pool2d(input=conv, pool_size=3, pool_stride=2, pool_padding=1, pool_type='max')

    for block in range(len(depth)):
        for i in range(depth[block]):
            if block == 2:
                if i == 0:
                    conv_name="res"+str(block+2)+"a"
                else:
                    conv_name="res"+str(block+2)+"b"+str(i)
            else:
                    conv_name="res"+str(block+2)+chr(97+i)
            conv = bottleneck_block(input=conv,
                                    num_filters=num_filters[block],
                                    stride=2 if i == 0 and block != 0 else 1,
                                    name=conv_name)

    pool = fluid.layers.pool2d(input=conv, pool_size=7, pool_type='avg', global_pooling=True)
    return pool


In [6]:
##定义输入层
image=fluid.layers.data(name='image',shape=[3,224,224],dtype='float32')
label=fluid.layers.data(name='label',shape=[1],dtype='int64')

In [7]:
##停止梯度下降
pool=resnet(image)
pool.stop_gradient=True

##创建主程序
base_model_program=fluid.default_main_program().clone()
model=fluid.layers.fc(input=pool,size=12,act='softmax')

In [8]:
##定义损失函数和准确率函数
cost=fluid.layers.cross_entropy(input=model,label=label)
avg_cost=fluid.layers.mean(cost)
acc=fluid.layers.accuracy(input=model,label=label)

##定义优化方法
optimizer=fluid.optimizer.AdamOptimizer(learning_rate=5e-4)
opts=optimizer.minimize(avg_cost)

##定义训练场所
use_gpu=True
place=fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe=fluid.Executor(place)

##进行参数初始化
exe.run(fluid.default_startup_program())

[]

In [9]:
##预训练模型路径
src_pretrain_model_path='work/pretrained/ResNet101_vd_pretrained'

##判断模型文件是否存在
def if_exit(var):
    path=os.path.join(src_pretrain_model_path,var.name)
    exist=os.path.exists(path)
    return exist

##加载模型文件，且只加载存在模型的模型文件
fluid.io.load_vars(executor=exe,dirname=src_pretrain_model_path,predicate=if_exit,main_program=base_model_program)

In [10]:
##优化内存
optimized = fluid.transpiler.memory_optimize(input_program=fluid.default_main_program(), print_log=False)

##定义数据维度
feeder=fluid.DataFeeder(place=place,feed_list=[image,label])

for pass_id in range(15):
    for batch_id,data in enumerate(train_reader()):
        train_cost,train_acc=exe.run(program = fluid.default_main_program(),feed=feeder.feed(data),fetch_list=[avg_cost,acc])
        if batch_id%50==0:
            print('Pass:%d, Batch:%d, Cost:%0.5f, Accuracy:%0.5f' %
                  (pass_id, batch_id, train_cost[0], train_acc[0]))

memory_optimize is deprecated. Use CompiledProgram and Executor


Pass:0, Batch:0, Cost:2.31926, Accuracy:0.18750
Pass:0, Batch:50, Cost:2.25434, Accuracy:0.25000
Pass:0, Batch:100, Cost:1.93801, Accuracy:0.37500
Pass:1, Batch:0, Cost:2.12956, Accuracy:0.25000
Pass:1, Batch:50, Cost:1.92962, Accuracy:0.37500
Pass:1, Batch:100, Cost:1.88695, Accuracy:0.62500
Pass:2, Batch:0, Cost:2.08787, Accuracy:0.18750
Pass:2, Batch:50, Cost:1.98429, Accuracy:0.31250
Pass:2, Batch:100, Cost:1.63930, Accuracy:0.50000
Pass:3, Batch:0, Cost:1.20482, Accuracy:0.68750
Pass:3, Batch:50, Cost:1.62702, Accuracy:0.50000
Pass:3, Batch:100, Cost:1.22755, Accuracy:0.68750
Pass:4, Batch:0, Cost:1.38590, Accuracy:0.56250
Pass:4, Batch:50, Cost:1.64612, Accuracy:0.50000
Pass:4, Batch:100, Cost:1.59459, Accuracy:0.50000
Pass:5, Batch:0, Cost:1.65650, Accuracy:0.50000
Pass:5, Batch:50, Cost:0.99064, Accuracy:0.75000
Pass:5, Batch:100, Cost:1.29896, Accuracy:0.68750
Pass:6, Batch:0, Cost:1.08770, Accuracy:0.75000
Pass:6, Batch:50, Cost:1.25853, Accuracy:0.62500
Pass:6, Batch:100, Co

In [11]:
save_pretrain_model_path='models/step-1_model/'

##删除旧的模型文件
shutil.rmtree(save_pretrain_model_path,ignore_errors=True)
##创建保存模型文件记录
os.makedirs(save_pretrain_model_path)
##保存参数模型
fluid.io.save_params(executor=exe,dirname=save_pretrain_model_path)

# Step-2

In [1]:
import os
import shutil
import paddle as paddle
import paddle.dataset.flowers as flowers
import paddle.fluid as fluid
from paddle.fluid.param_attr import ParamAttr
import reader

In [2]:
# 获取数据
train_reader = paddle.batch(reader.train(), batch_size=16)
test_reader = paddle.batch(reader.val(), batch_size=16)

In [3]:
##定义resnet，加上fc
def resnet(input,class_dim):
    def conv_bn_layer(input, num_filters, filter_size, stride=1, groups=1, act=None, name=None):
        conv = fluid.layers.conv2d(input=input,
                                   num_filters=num_filters,
                                   filter_size=filter_size,
                                   stride=stride,
                                   padding=(filter_size - 1) // 2,
                                   groups=groups,
                                   act=None,
                                   param_attr=ParamAttr(name=name + "_weights"),
                                   bias_attr=False,
                                   name=name + '.conv2d.output.1')
        if name == "conv1":
            bn_name = "bn_" + name
        else:
            bn_name = "bn" + name[3:]
        return fluid.layers.batch_norm(input=conv,
                                       act=act,
                                       name=bn_name + '.output.1',
                                       param_attr=ParamAttr(name=bn_name + '_scale'),
                                       bias_attr=ParamAttr(bn_name + '_offset'),
                                       moving_mean_name=bn_name + '_mean',
                                       moving_variance_name=bn_name + '_variance', )

    def shortcut(input, ch_out, stride, name):
        ch_in = input.shape[1]
        if ch_in != ch_out or stride != 1:
            return conv_bn_layer(input, ch_out, 1, stride, name=name)
        else:
            return input
            
    def bottleneck_block(input, num_filters, stride, name):
        conv0 = conv_bn_layer(input=input,
                              num_filters=num_filters,
                              filter_size=1,
                              act='relu',
                              name=name + "_branch2a")
        conv1 = conv_bn_layer(input=conv0,
                              num_filters=num_filters,
                              filter_size=3,
                              stride=stride,
                              act='relu',
                              name=name + "_branch2b")
        conv2 = conv_bn_layer(input=conv1,
                              num_filters=num_filters * 4,
                              filter_size=1,
                              act=None,
                              name=name + "_branch2c")

        short = shortcut(input, num_filters * 4, stride, name=name + "_branch1")

        return fluid.layers.elementwise_add(x=short, y=conv2, act='relu', name=name + ".add.output.5")

    depth = [3, 4, 23, 3]
    num_filters = [64, 128, 256, 512]

    conv = conv_bn_layer(input=input, num_filters=64, filter_size=7, stride=2, act='relu', name="conv1")
    conv = conv_bn_layer(input=input, num_filters=32, filter_size=3, stride=2, act='relu', name='conv1_1')
    conv = conv_bn_layer(input=conv, num_filters=32, filter_size=3, stride=1, act='relu', name='conv1_2')
    conv = conv_bn_layer(input=conv, num_filters=64, filter_size=3, stride=1, act='relu', name='conv1_3')
    conv = fluid.layers.pool2d(input=conv, pool_size=3, pool_stride=2, pool_padding=1, pool_type='max')

    for block in range(len(depth)):
        for i in range(depth[block]):
            if block == 2:
                if i == 0:
                    conv_name="res"+str(block+2)+"a"
                else:
                    conv_name="res"+str(block+2)+"b"+str(i)
            else:
                    conv_name="res"+str(block+2)+chr(97+i)
            conv = bottleneck_block(input=conv,
                                    num_filters=num_filters[block],
                                    stride=2 if i == 0 and block != 0 else 1,
                                    name=conv_name)

    pool = fluid.layers.pool2d(input=conv, pool_size=7, pool_type='avg', global_pooling=True)
    output=fluid.layers.fc(input=pool,size=class_dim,act='softmax')
    return output

In [4]:
##定义输入层
image = fluid.layers.data(name='image', shape=[3, 224, 224], dtype='float32')
label = fluid.layers.data(name='label', shape=[1], dtype='int64')

##获取分类器
model = resnet(image,12)

In [5]:
##获取损失函数和准确率函数
cost = fluid.layers.cross_entropy(input=model, label=label)
avg_cost = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input=model, label=label)

In [6]:
##获取训练和测试程序
test_program = fluid.default_main_program().clone(for_test=True)

In [7]:
##定义优化方法
optimizer=fluid.optimizer.AdamOptimizer(learning_rate=5e-4)
opts=optimizer.minimize(avg_cost)

In [8]:
##定义一个使用GPU的执行器
use_gpu=False
place=fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe=fluid.Executor(place)

##进行参数初始化
exe.run(fluid.default_startup_program())

[]

In [9]:
##经过step-1处理后的的预训练模型
pretrained_model_path = 'models/step-1_model/'

##加载经过处理的模型
fluid.io.load_params(executor=exe, dirname=pretrained_model_path)

In [13]:
##定义数据维度
feeder=fluid.DataFeeder(place=place,feed_list=[image,label])
for pass_id in range(10):
    ##训练
    for batch_id,data in enumerate(train_reader()):
        train_cost,train_acc=exe.run(program=fluid.default_main_program(),feed=feeder.feed(data),fetch_list=[avg_cost,acc])
        if batch_id%100==0:
            print('Pass:%d, Batch:%d, Cost:%0.5f, Accuracy:%0.5f' %
                  (pass_id, batch_id, train_cost[0], train_acc[0]))
    ##测试
    test_acc=[]
    test_costs=[]
    for batch_id,data in enumerate(test_reader()):
        test_cost,test_acc=exe.run(program=test_program,feed=feeder.feed(data), fetch_list=[avg_cost,acc])
        test_acc.append(test_acc[0])
        test_costs.append(test_cost[0])
    test_cost = (sum(test_costs) / len(test_costs))
    test_acc = (sum(test_accs) / len(test_accs))
    print('Test:%d, Cost:%0.5f, Accuracy:%0.5f' % (pass_id, test_cost, test_acc))
    

In [ ]:
##保存预测模型
save_path = 'models/step_2_model/'

##删除旧的模型文件
shutil.rmtree(save_path, ignore_errors=True)
##创建保持模型文件目录
os.makedirs(save_path)
##保存预测模型
fluid.io.save_inference_model(save_path, feeded_var_names=[image.name], target_vars=[model], executor=exe)